# This is the main notebook for interrogating the results of models as they train

In [ ]:
## import integrated_cell.SimpleLogger as SimpleLogger
import torch
from IPython.core.display import display
import pickle
import os
import PIL.Image
import numpy as np
import scipy.misc as misc
import glob
import json
from natsort import natsorted

%matplotlib inline

class RenamingUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'integrated_cell.SimpleLogger':
            module = 'integrated_cell.simplelogger'
        return super().find_class(module, name)


import warnings
warnings.filterwarnings("ignore")

dirs_to_search = [
        
#                     '/allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/*',
#                     '/allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_beta_ref/*',
                    './training_scripts/*/'
    
                ]

model_dirs = list()

for my_dir in dirs_to_search:
    model_dirs += glob.glob(my_dir)

model_dirs = natsorted(model_dirs)[::-1]    
    

def show_dir(model_dir):    
    logger_file = '{0}/logger_tmp.pkl'.format(model_dir)
    
    if not os.path.exists(logger_file):
        print('Could not find logger at ' + logger_file)
        return

    print(model_dir)
    
    try:
        logger = RenamingUnpickler(open( logger_file, "rb" )).load()
#         logger = pickle.load( open( logger_file, "rb" ) )
        epoch = max(logger.log['epoch'])
    except:
        pass
    
    try:
        opt = pickle.load( open( '{0}/opt.pkl'.format(model_dir), "rb" ))
    except:
        opt = json.load(open( '{0}/../args.json'.format(model_dir), "rb" ))

    print(opt)
    print('Epoch: ' + str(epoch))

    im_progress_path = '{0}/progress_{1}.png'.format(model_dir, int(epoch))
    im_progress = misc.imread(im_progress_path)

    im_history_path = '{0}/history.png'.format(model_dir)
    im_history = misc.imread(im_history_path)    

    im_history_short_path = '{0}/history_short.png'.format(model_dir)
    im_history_short = misc.imread(im_history_short_path)    

    im_embedding_path = '{0}/embedding.png'.format(model_dir)
    im_embedding = misc.imread(im_embedding_path)        

    display(PIL.Image.fromarray(im_progress))
    display(PIL.Image.fromarray(np.concatenate((im_history, im_history_short, im_embedding), 1)))
    print(' ')
    
for model_dir in model_dirs:
    
    sub_dirs = glob.glob(os.path.join(model_dir, 'ref_model/'))
    
    for sub_dir in sub_dirs:
        try:
            show_dir(sub_dir)
        except:
            print('could not load ' + model_dir)
            